In [287]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC


In [288]:
train_data = pd.read_csv("datasets/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [289]:
# prepare data
train_data["Age_decade"] = round(train_data["Age"] / 10)
# print(train_data[["Age", "Age_decade"]])

features = ["Pclass", "Sex", "SibSp", "Parch", "Embarked", "Age_decade"]
train_data.dropna(subset=features, inplace=True, axis="rows")
# train_data["Age"] = train_data["Age"].round().astype(int)
train_data.describe()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Age_decade
count,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000
mean,448.589888,0.404494,2.240169,29.642093,0.514045,0.432584,34.567251,2.946629
std,258.683191,0.491139,0.836854,14.492933,0.930692,0.854181,52.938648,1.482183
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,222.750000,0.000000,1.000000,20.000000,0.000000,0.000000,8.050000,2.000000
50%,445.000000,0.000000,2.000000,28.000000,0.000000,0.000000,15.645850,3.000000
75%,677.250000,1.000000,3.000000,38.000000,1.000000,1.000000,33.000000,4.000000
max,891.000000,1.000000,3.000000,80.000000,5.000000,6.000000,512.329200,8.000000


In [290]:
# print(np.any(np.isposinf(train_data["Age"])))
# print(np.any(np.isneginf(train_data["Age"])))
# print(np.where(np.isnan(train_data["Age"])))
# train_data["Age"].describe()
# train_data["Age"].to_csv("ages.csv", index = False)

In [291]:
test_data = pd.read_csv("datasets/test.csv")
test_data.fillna(value={"Age": test_data["Age"].median()}, inplace=True, axis="rows")
test_data["Age_decade"] = round(test_data["Age"] / 10)
if test_data[features].isnull().values.any():
    raise ValueError("Test data features contain null value")
# test_data.dropna(subset=features, inplace=True, axis="rows")
test_data.describe()


,PassengerId,Pclass,Age,SibSp,Parch,Fare,Age_decade
count,418.000000,418.000000,418.000000,418.000000,418.000000,417.000000,418.000000
mean,1100.500000,2.265550,29.599282,0.447368,0.392344,35.627188,3.011962
std,120.810458,0.841838,12.703770,0.896760,0.981429,55.907576,1.303877
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,23.000000,0.000000,0.000000,7.895800,2.000000
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200,3.000000
75%,1204.750000,3.000000,35.750000,1.000000,0.000000,31.500000,4.000000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200,8.000000


In [292]:
y = train_data["Survived"]
X = pd.get_dummies(train_data[features])
# print(X.describe())
X_test = pd.get_dummies(test_data[features])
# classifier = SVC(kernel="poly", C=0.025)
# classifier = DecisionTreeClassifier(max_depth=5, random_state=1)
# classifier = KNeighborsClassifier()
classifier = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,
    random_state=1
)
classifier.fit(X, y)
predictions = classifier.predict(X_test)


In [293]:

submission_data = pd.DataFrame({"PassengerId": test_data.PassengerId, "Survived": predictions})
submission_data.to_csv("datasets/submission.csv", index=False)
